In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import namedtuple, deque

In [3]:
import subprocess
import os
import pandas as pd
import time 

In [30]:

class ServerEnv:
    def __init__(self):
        self.core_data = None  
        self.current_state = None 
        self.VNFs = ["l2fwd", "l3fwd"]
        self.pcm_path = "/home/qiong/resource_contention/scripts/"
        self.measurement_time = 30 
        self.experiment_duration=2100

    def reset(self):
        # Reset PQoS
        subprocess.run(['sudo', 'pqos', '-R'], check=True)

        # Assuming processes_to_kill is defined somewhere in your code
        for process in processes_to_kill:
            try:
                pids = subprocess.check_output(['pgrep', '-f', process]).decode().strip().split('\n')
                for pid in pids:
                    subprocess.run(['sudo', 'kill', pid], check=True)
            except subprocess.CalledProcessError:
                # If pgrep didn't find the process, ignore it
                pass

        # CSV file reset
        csv_file_path = "/home/qiong/resource_contention/scripts/ddqnonline.csv"
        if os.path.exists(csv_file_path):
            os.remove(csv_file_path)

        # No need to check if file exists here, as we're going to create or overwrite it anyway
        with open(csv_file_path, 'w') as f:
            # If you need to write initial content to the file, do it here
            pass

        # Initialize
        self.current_state = [0, 0, 0, 0, 0, 0, 0, 0]
        # If you need to set the state to something specific at reset, do it before returning
        return self.current_state
    
    def llc_way_to_param(self,llc_way):
        bitnary_str = '1'*llc_way
        current_config=self.get_current_llc_config()
        for i in range(len(curerent_config) - len(binary_str),-1,-1):
            if current_config[i:i+len(binary_str)] =='0'*len(binary_str):
                new_config = current_config[:i] + binary_str + current_config[i+len(binary_str):]
                break
        else:
            raise Exception('No available space for the requested LLC way')
        new_config_hex = hex(int(new_config,2))
        
        return new_config_hex
    
    def update_core_cos_mapping(self):
        #  pqos -s 
        output = subprocess.check_output(['sudo', 'pqos', '-s'], text=True)
        # cos-core-map
        core_cos_mapping = {}
        for line in output.splitlines():
            match = re.search(r'Core (\d+), L2ID \d+, L3ID \d+ => COS(\d+)', line)
            if match:
                core = int(match.group(1))
                cos = int(match.group(2))
                core_cos_mapping[core] = cos
        # cos-core-mapping
        self.core_cos_mapping = core_cos_mapping 
        self.update_cos_configurations()
        
    def get_cos_to_llc_mapping():

        result = subprocess.run(['sudo', 'pqos', '-s'], capture_output=True, text=True)
        cos_llc_map = {}
        cos_llc_matches = re.findall(r'L3CA COS(\d+) => MASK (\w+)', result.stdout)
        for match in cos_llc_matches:
            cos_id, llc_mask = match
            cos_llc_map[int(cos_id)] = llc_mask

        return cos_llc_map
    
    def get_current_llc_config(self):
        self.update_core_cos_mapping()
        cos_llc_map = self.get_cos_to_llc_mapping()
        
        # core to llc
        core_llc_configuration = {}
        for core, cos in self.core_cos_mapping.items():
            if cos in cos_llc_map:
                llc_mask = cos_llc_map[cos]
                core_llc_configuration[core] = llc_mask
            else:
                core_llc_configuration[core] = 'Unknown' 

        return core_llc_configuration
    
    def step(self, action):
        llc_way = action.get('llc_way')
        if llc_way not in range(1, 21): 
            print(f"Invalid LLC way: {llc_way}. Must be between 1 and 20.")
            return
        param = self.llc_way_to_param(llc_way)
        command = f"sudo pqos -e 'llc:{llc_way}={param}'"
        try: 
            subprocess.run(command,shell=Ture,check=True)
        except subprocess.CalledProcessError as e:
            print(f"Failed to set LLC allocation: {e}")
            
        self.collect_data()
        reward = self.calculate_reward()
        done = self.check_if_done()
        return self.current_state, reward, done

    def collect_data(self):
        VNFs = ["l2fwd", "l3fwd"]

        for val in VNFs:
            print(f"VNF: {val}")
            try:
                pid = subprocess.check_output(['pidof', val]).decode().strip()
                core = subprocess.check_output(['ps', '-mo', 'psr', pid]).decode().splitlines()[-1].strip()
                print(f"VNF {val} runs on Core {core}")
                pcm_command = f"sudo {self.pcm_path}pcm -silent -nsys -yc {core} -csv='{val}-pcm.csv'"
                subprocess.Popen(pcm_command, shell=True)
            except subprocess.CalledProcessError:
                print(f"Skip non-existing VNF {val}")

        # Global data: pcie & pcm-memory
        subprocess.Popen(f"sudo {self.pcm_path}pcm-pcie -csv='pcm-pcie.csv'", shell=True)
        subprocess.Popen(f"sudo {self.pcm_path}pcm-memory -csv='pcm-memory.csv'", shell=True)

        print("Collecting data...")
        subprocess.run(['sleep', str(self.measurement_time)])

        # End all processing
        subprocess.run(['sudo', 'killall', 'pcm'])
        print("Measurement finished!")

    def calculate_reward(self):

        throughput = self.core_data['throughput']
        latency = self.core_data['latency']
        current_llc_bin = bin(int(self.current_llc_config, 16))[2:]  
        llc_occupation = current_llc_bin.count('1') 
        llc_occupation_normalized = llc_occupation / 20.0  
        reward = (throughput * throughput - latency / 100000) - llc_occupation_normalized

        return reward

    def check_if_done(self):
        current_time = time.time()
        if current_time - self.start_time >= self.experiment_duration:
            return True 
        else:
            return False 


In [31]:
env = ServerEnv()


In [32]:
# Define the DDQN network
class Net(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = self.relu(self.fc1(state))
        return self.fc2(x)

In [33]:
# Experience replay memory
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(namedtuple("Transition", ("state", "action", "next_state", "reward"))(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [34]:
# Function to select an action
def select_action(state, online_net, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            return online_net(state).max(1)[1].data.numpy()
        action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE) 
        exploit_count +=1
    else:
        return torch.tensor([[random.randrange(action_size)]], dtype=torch.long)
        explore_count +=1
    return action

# Training process

In [35]:
def train_ddqn(env,online_net,target_net, optimizer, memory, episodes, batch_size, target_update):
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        epsilon = max(epsilon_end, epsilon_start * (epsilon_decay ** episode))
        
        # Update the target network
        if episode % target_update == 0:
            target_net.load_state_dict(online_net.state_dict())
            
        for t in count():
            action = action = select_action(state, online_net, epsilon)
            next_state, reward, done, _ = env.step(action)
            memory.push(state,action,next_state,reward)
            
            if len(memory) > batch_size:
                transitions = memory.sample(batch_size)
                batch = namedtuple("Transition", ("state", "action", "next_state", "reward"))(*zip(*transitions))
                # Prepare the batch for training
                non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
                non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
                state_batch = torch.cat(batch.state)
                action_batch = torch.cat(batch.action)
                reward_batch = torch.tensor(batch.reward)

                # Compute Q values
                state_action_values = online_net(state_batch).gather(1, action_batch)
                next_state_values = torch.zeros(batch_size)
                next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
                expected_state_action_values = (next_state_values * gamma) + reward_batch

                # Compute loss
                loss = nn.functional.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
                
                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if done:
                break

# Running Process

In [36]:
def run_ddqn(env, online_net):
    state = env.reset()
    total_reward = 0
    while True:
        action = online_net(state).max(1)[1].view(1,1)
        state, reward, done, _ = env.step(action)
        total_reward +=reward
        if done:
            break
    return total_reward


In [6]:
import random

def generate_ddio_allocation(current_ddio_hex):
    # Convert hex to binary, ensuring it's padded to represent 20 bits
    current_ddio_bin = bin(int(current_ddio_hex, 16))[2:].zfill(20)
    
    # Count occupied and unoccupied bits
    occupied = current_ddio_bin.count('1')
    unoccupied = 20 - occupied
    
    # Randomly decide to increase (by b) or decrease (by a) the allocation
    a = random.randint(0, occupied)  # Possible decrease
    b = random.randint(0, unoccupied)  # Possible increase
    
    # Generating new binary DDIO configuration with left-to-right allocation or deallocation
    new_ddio_bin = list(current_ddio_bin)
    if b > 0 and occupied < 20:  # If there's room to increase
        added = 0
        for i in range(20):
            if new_ddio_bin[i] == '0' and added < b:
                new_ddio_bin[i] = '1'
                added += 1
            if added >= b:
                break
    elif a > 0:  # If we need to decrease
        removed = 0
        for i in range(19, -1, -1):
            if new_ddio_bin[i] == '1' and removed < a:
                new_ddio_bin[i] = '0'
                removed += 1
            if removed >= a:
                break
    
    # Convert back to hex
    new_ddio_hex = hex(int(''.join(new_ddio_bin), 2)).upper().replace('0X', '0x').zfill(5)
    
    return new_ddio_hex

# Example usage
current_ddio_hex = "C0000"
new_ddio_hex = generate_ddio_allocation(current_ddio_hex)
print(f"Current DDIO setting: {current_ddio_hex} -> New DDIO setting: {new_ddio_hex}")




Current DDIO setting: C0000 -> New DDIO setting: 0xFF000


In [11]:

# Example usage
current_ddio_hex = "C0000"
new_ddio_hex = generate_ddio_allocation(current_ddio_hex)
print(f"Current DDIO setting: {current_ddio_hex} -> New DDIO setting: {new_ddio_hex}")


Current DDIO setting: C0000 -> New DDIO setting: 0xFFF00
